In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
df_lkp = pd.read_csv('taxi_zone_lookup.csv')
df_trip = pd.read_parquet('green_tripdata_2025-11.parquet')

In [3]:
len(df_trip)

46912

In [19]:
df_trip = df_trip.merge(df_lkp, how = 'inner', left_on = 'PULocationID', right_on = 'LocationID')
df_trip = df_trip.merge(df_lkp, how = 'inner', left_on = 'DOLocationID', right_on = 'LocationID', suffixes = ('_pu','_do'))

df_trip.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,congestion_surcharge,cbd_congestion_fee,LocationID_pu,Borough_pu,Zone_pu,service_zone_pu,LocationID_do,Borough_do,Zone_do,service_zone_do
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone,42,Manhattan,Central Harlem North,Boro Zone
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone,42,Manhattan,Central Harlem North,Boro Zone
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.00,0.0,83,Queens,Elmhurst/Maspeth,Boro Zone,160,Queens,Middle Village,Boro Zone
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.00,0.0,166,Manhattan,Morningside Heights,Boro Zone,127,Manhattan,Inwood,Boro Zone
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,2.75,0.0,166,Manhattan,Morningside Heights,Boro Zone,262,Manhattan,Yorkville East,Yellow Zone


In [ ]:
def ingest_df(
    df: pd.DataFrame,
    engine,
    target_table: str,
    chunksize: int = 100_000,
):
    # 1) create table
    df.head(0).to_sql(
        name=target_table,
        con=engine,
        if_exists="replace",
        index=False
    )
    print(f"Table {target_table} created")

    # 2) add data
    total = len(df)
    for i in range(0, total, chunksize):
        chunk = df.iloc[i:i+chunksize]
        chunk.to_sql(
            name=target_table,
            con=engine,
            if_exists="append",
            index=False
        )
        print(f"Inserted rows: {min(i+chunksize, total)}/{total}")

    print(f"Done ingesting to {target_table}")


In [ ]:
ingest_df(
    df=df_trip,
    engine=engine,
    target_table="green_taxi_2025_11_merged",
    chunksize=100_000
)